In [28]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
import numpy as np
import findspark
findspark.init("/opt/cloudera/parcels/CDH-6.3.1-1.cdh6.3.1.p0.1470567/lib/spark","/usr/bin/python2.7")

import os
os.environ["JAVA_HOME"] = "/usr/java/jdk1.8.0_181-cloudera"

from pyspark.sql import functions as F
from pyspark import SparkContext
from pyspark.sql import SparkSession,HiveContext,Window
from pyspark.sql.types import IntegerType, FloatType, DoubleType, ArrayType, StringType, DecimalType
from pyspark.sql.functions import *
import re
import datetime
from pychattr.channel_attribution import MarkovModel

spark_session = SparkSession.builder.enableHiveSupport().appName("artefact_attribution_analysis") \
    .config("spark.driver.memory","10g") \
    .config("spark.pyspark.driver.python","/usr/bin/python2.7")\
    .config("spark.pyspark.python","/usr/bin/python2.7") \
    .config("spark.yarn.executor.memoryOverhead","8G") \
    .getOrCreate()
hc = HiveContext(spark_session.sparkContext)

In [2]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import itertools

In [3]:
names = ['mobile','touchpoint_id','action_time','row_number','time_lag','tp_id_through','row_number_lag','conv_period']
df = pd.read_csv('data/tp_analysis_base.csv',sep = '\t', names = names)
df['mobile'] = df['mobile'].astype(str)

In [5]:
# fetch id mapping system
#id_mapping = hc.sql('select * from marketing_modeling.dw_touchpoints_id_system').toPandas()

In [6]:
id_mapping = pd.read_csv('data/id_mapping.csv')

tp_name = {k:v for k,v in zip(id_mapping.touchpoint_id,id_mapping.touchpoint_name)}
#inbound_map = {k:v for k,v in zip(id_mapping.touchpoint_id,id_mapping.is_inbound)}

In [ ]:
len(df.mobile.unique())

##### 5.2.3 参与活动前后三个触点流转比例 - 线下活动

In [ ]:
# get activity_index
## offline
offline_act_user = df[df.touchpoint_id.str.startswith('008001')].mobile.unique().tolist()
offline_act = df[df.mobile.isin(offline_act_user)][['mobile','touchpoint_id','row_number','action_time','segment']]
def act_id(tp_id):
    if tp_id.startswith('008001'):
        return tp_id[:9]+'000_tp'
    else:
        return tp_id
offline_act['act_id'] = offline_act['touchpoint_id'].apply(lambda x:act_id(x))

In [ ]:
offline_act_index = offline_act[offline_act.act_id.str.startswith('008001')][['mobile','act_id','row_number']].groupby(by=['mobile','act_id'],as_index=False).agg({'row_number':'min'})

offline_act_index = offline_act_index.merge(offline_act,on=['mobile','row_number','act_id'],how='inner')

offline_act_index = offline_act_index.rename(columns={'row_number':'act_index',
                                                      'action_time':'act_time'})[['mobile','act_index','act_id','segment','act_time']]

offline_act_df = offline_act_index.merge(offline_act[['mobile','touchpoint_id','row_number','action_time']],on='mobile',how = 'inner')

In [ ]:
lag_ = [-3,-2,-1,1,2,3]
offline_final_result = pd.DataFrame()
for lag in lag_:
    lag_df = offline_act_df[(offline_act_df.act_index < offline_act_df.row_number+lag) \
                            & (offline_act_df.action_time > offline_act_df.act_time)\
                            & (offline_act_df.touchpoint_id.isin(['001001001000_tp', '001001002000_tp']) == False)]
    
    tmp_lag_df = lag_df.groupby(by=['mobile','act_id'],as_index=False).agg({'row_number':'min'})
    if lag > 0:
        tmp_lag_df['row_number'] = tmp_lag_df['row_number'].apply(lambda x:x+lag-1)
    else:
        pass
    final_df = offline_act_index[['mobile','act_id','segment']].merge(tmp_lag_df,on=['mobile','act_id'],how='left')\
    .merge(lag_df,on=['mobile','act_id','row_number'],how='left')

    final_df['diff'] = (pd.to_datetime(final_df.action_time) - pd.to_datetime(final_df.act_time)).apply(lambda x:x.total_seconds()/3600)
    #.groupby(by=['act_id','segment_x']).touchpoint_id.value_counts()

    final_df['touchpoint_id'] = final_df['touchpoint_id'].fillna('Exit')
    final_df['is_inbound'] = final_df['touchpoint_id'].apply(lambda x:is_inbound(x))

    result = final_df.groupby(by=['act_id','touchpoint_id'],as_index=False).agg({'mobile':'count','diff':'median'})
    #result = final_df.groupby(by=['act_id','is_inbound'],as_index=False).agg({'mobile':'count','diff':'median'})

   # result = result.merge(id_mapping[['level_1_name','level_2_name','level_3_name','level_4_name',
   #                                   'touchpoint_id','touchpoint_level']],on='touchpoint_id',how='left')
    if lag > 0:
        prefix = 'Next_'
    else:
        prefix = 'Previous_'
    result['type'] = prefix+str(np.abs(lag))
    offline_final_result = offline_final_result.append(result)

#offline_final_result.to_csv('offline_act_tp_transition.csv',index=False)

##### 5.2.3 参与活动前后三个触点流转比例 - 线上活动

In [ ]:
# online
adhoc_app_act = hc.sql('select * from marketing_modeling.dw_ts_adhoc_app_activity_i').toPandas()
adhoc_app_act.mobile = adhoc_app_act.mobile.astype(str)

In [175]:
online_act_tp = ['008002006005_tp','008002006005_tp','008002006004_tp','008002006002_tp','008002006003_tp',
                 '008002002000_tp','008002002001_tp','008002002002_tp','008002002003_tp','008002002004_tp',
                 '008002002005_tp','008002002006_tp','008002002007_tp','008002002008_tp','008002006001_tp',
                 '008002003000_tp','008002003001_tp','008002003002_tp','008002003003_tp','008002006005_tp',
                 '008002006005_tp']

online_act_user = df[df.touchpoint_id.isin(online_act_tp)].mobile.unique().tolist()
online_act = df[df.mobile.isin(online_act_user)][['mobile','touchpoint_id','row_number','action_time']]

online_act = online_act.merge(adhoc_app_act[['mobile','touchpoint_id','activity_name','activity_type']],on=['mobile','touchpoint_id'],how='left')

In [176]:
online_act['act_type'] = np.nan
online_act['act_type'][online_act.touchpoint_id.str.startswith('008002002')] = 'APP直播'
online_act['act_type'][online_act.touchpoint_id.isin(['008002006004_tp','008002006002_tp','008002006003_tp'])] = '社区活动'
online_act['act_type'][online_act.touchpoint_id.isin(['008002006001_tp'])] = '推荐或裂变' #'008002003000_tp','008002003001_tp','008002003002_tp','008002003003_tp'
online_act['act_type'][(online_act.touchpoint_id == '008002006005_tp') & (online_act.activity_name == '推荐有礼')] = '推荐或裂变'
online_act['act_type'][(online_act.touchpoint_id == '008002006005_tp') & (online_act.activity_type == 'Cyberster')] = '众创'
online_act['act_type'][(online_act.touchpoint_id == '008002006005_tp') & (online_act.activity_type == '集点积分')] = '集点积分'
online_act['act_type'][(online_act.touchpoint_id == '008002006005_tp')] = online_act['act_type'][(online_act.touchpoint_id == '008002006005_tp')].fillna('其他活动')

/app/anaconda3/envs/python37_smcsandbox/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
online_act_index = online_act[online_act.act_type.isna()==False][['mobile','row_number','act_type']].groupby(by=['mobile','act_type'],as_index=False).agg({'row_number':'min'})

online_act_index = online_act_index.merge(online_act,on=['mobile','row_number','act_type'],how='inner')

online_act_index = online_act_index.rename(columns={'row_number':'act_index',
                                                      'action_time':'act_time'})[['mobile','act_index','act_type','segment','act_time']]

online_act_df = online_act_index.merge(online_act[['mobile','touchpoint_id','row_number','action_time']],on='mobile',how = 'inner')

In [ ]:
def is_inbound(x):
    if x.startswith('014'):
        return '战败操作'
    elif x == 'Exit':
        return 'Exit'
    elif x.startswith('001'):
        return '1'
    else:
        return inbound_map[x]

In [ ]:
lag_ = [-3,-2,-1,1,2,3]
final_result = pd.DataFrame()
for lag in lag_:
    lag_df = online_act_df[(online_act_df.act_index < online_act_df.row_number+lag) \
                            & (online_act_df.action_time > online_act_df.act_time)\
                            & (online_act_df.touchpoint_id.isin(['001001001000_tp', '001001002000_tp']) == False)]
 
    tmp_lag_df = lag_df.groupby(by=['mobile','act_type'],as_index=False).agg({'row_number':'min'})    
    if lag > 0:
        tmp_lag_df['row_number'] = tmp_lag_df['row_number'].apply(lambda x:x+lag-1)
    else:
        pass
    final_df = online_act_index[['mobile','act_type','segment']].merge(tmp_lag_df,on=['mobile','act_type'],how='left')\
    .merge(lag_df,on=['mobile','act_type','row_number'],how='left')
    
    final_df['diff'] = (pd.to_datetime(final_df.action_time) - pd.to_datetime(final_df.act_time)).apply(lambda x:x.total_seconds()/3600)

    final_df['touchpoint_id'] = final_df['touchpoint_id'].fillna('Exit')
    final_df['is_inbound'] = final_df['touchpoint_id'].apply(lambda x:is_inbound(x))

    result = final_df.groupby(by=['act_type','touchpoint_id'],as_index=False).agg({'mobile':'count','diff':'median'})
    #result = final_df.groupby(by=['act_type','is_inbound'],as_index=False).agg({'mobile':'count','diff':'median'})
    
    #result = result.merge(id_mapping[['level_1_name','level_2_name','level_3_name','level_4_name',
    #                                  'touchpoint_id','touchpoint_level']],on='touchpoint_id',how='left')
    if lag > 0:
        prefix = 'Next_'
    else:
        prefix = 'Previous_'
    result['type'] = prefix+str(np.abs(lag))
    final_result =final_result.append(result)

#final_result.to_csv('result/online_act_tp_transition.csv',index=False)

In [ ]:
offline_final_result['act_id'] = offline_final_result['act_id'].apply(lambda x:tp_name[x])

In [ ]:
result_vf = pd.DataFrame()
for act in offline_final_result['act_id'].unique():
    for stage in ['Previous_1','Previous_2','Previous_3','Next_1','Next_2','Next_3']:
        print('Processing:{0}---{1}'.format(act,stage))
        tmp_data = offline_final_result[(offline_final_result.act_id == act) & (offline_final_result.type == stage)]
        tmp_result = tmp_data.groupby(by='touchpoint_id',as_index=False).agg({'mobile':'sum'})
        tmp_result['percentage'] = tmp_result['mobile'].apply(lambda x: x/tmp_data.mobile.sum())
        tmp_result['type'] = act+'&'+stage
        tmp_result = tmp_result.sort_values(by='percentage',ascending=False)
        tmp_result = tmp_result[:3]
        tmp_result['rank_'] = [1,2,3]
        result_vf = result_vf.append(tmp_result)

In [ ]:
result_vf = pd.DataFrame()
for act in final_result['act_type'].unique():
    for stage in ['Previous_1','Previous_2','Previous_3','Next_1','Next_2','Next_3']:
        print('Processing:{0}---{1}'.format(act,stage))
        tmp_data = final_result[(final_result.act_type == act)&(final_result.type == stage)]
        tmp_result = tmp_data.groupby(by = 'touchpoint_id',as_index=False).agg({'mobile':'sum'})
        tmp_result['percentage'] = tmp_result['mobile'].apply(lambda x: x/tmp_data.mobile.sum())
        tmp_result['type'] = act+'&'+stage
        tmp_result = tmp_result.sort_values(by='percentage',ascending=False)
        tmp_result = tmp_result[:3]
        tmp_result['rank_'] = [1,2,3]
        result_vf = result_vf.append(tmp_result)

In [ ]:
result_vf.merge(id_mapping[['level_1_name','level_2_name','level_3_name','level_4_name',
                                      'touchpoint_id','touchpoint_level']],on='touchpoint_id',how='left').to_csv('result/off_act_final.csv',index=False)